# Chapter 4 - Full syndromic dataset (pairwise)

## Setup

Run Python file to import libraries and custom functions

In [1]:
%run libraries.py

Using TensorFlow backend.
/Users/joefarrington/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/joefarrington/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Set paths for data and models

In [2]:
data_path = '/Users/joefarrington/Documents/MSc Machine Learning/Project/August/Data for import/'
results_path = '/Users/joefarrington/Documents/MSc Machine Learning/Project/August/Results/Ch4Allpairwise/'

Define a string to be added to the name of all saved results from this notebook

In [3]:
nbname = 'Ch4Allpairwise'

Import the data

In [4]:
X_all = pd.read_csv(data_path+'X_all_Rev.csv', index_col=0)
y_all = np.array(pd.read_csv(data_path+'y_all_Rev.csv', index_col=0, header=None)).reshape(-1,)

Define an sklearn scorer object to calculate the Matthews Correlation Coefficient (MCC)

In [5]:
mcc_scorer = make_scorer(matthews_corrcoef)

Define a named tuple to store the results

Create a list that contains the labels

In [7]:
B_labels = np.unique(y_all)
B_labels[1] = 'DS'
B_labels

array(['22q11DS', 'DS', 'FXS-F', 'FXS-M', 'NF1', 'PW-UPD', 'PW-del',
       'SMC15', 'TSC', 'XXY'], dtype=object)

Define a function to get pairwise data

In [10]:
def get_pairwise_data(class1, class2, X, y):
    
    """
    Returns the training data for a pairwise comparison
    
    Inputs
    class1 - a class label
    class2 - a class label
    X - Pandas dataframe of training data
    y - NumPy array of training labels
    
    Outputs
    X_c - Pandas dataframe of training data for class1 and class2 only
    y_c - NumPy array of training labels for class1 and class2 only
    """
    
    y_c = y[(y==class1) | (y == class2)]
    X_c = X[(y==class1) | (y ==class2)]
    
    return X_c, y_c

Get the labels (as they are in y) and all of the pairwise combinations

In [11]:
labels = np.unique(y_all)

In [12]:
comparisons = list(itertools.combinations(labels, 2))

Set the number of repeats (train/test splits) and fold (for cross-validation)

In [24]:
n_repeats = 10
n_folds = 10

## Model 1: OvO SVM with RBF kernel

Define a subfolder to store the results

In [40]:
subfolder = 'SVM_RBF/'

Define the model

In [41]:
svc_rbf = SVC(decision_function_shape='ovo', kernel='rbf')
steps_rbf = [('scaler', StandardScaler()),
     ('model',svc_rbf)]
pipeline_rbf = Pipeline(steps_rbf)

Define the parameters for the grid search

In [42]:
rbf_C_range = np.logspace(start=-5, stop=15, num=21, base=2)
rbf_gamma_range = np.logspace(start=-15, stop=3, num=19, base=2)
rbf_parameters = dict(model__gamma=rbf_gamma_range, model__C=rbf_C_range)

Create dataframes to store the pairwise results

In [43]:
mean_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

mean_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

Train the pairwise models and store the results

In [44]:
overall_res = {}

for comp in comparisons:
    
    X_c, y_c = get_pairwise_data(comp[0], comp[1], X_all, y_all)
    results_dict = repeated_training(X_c, y_c, n_repeats, n_folds, rbf_parameters, pipeline_rbf, mcc_scorer, verbose=1)
    
    rkeys = [x for x in results_dict.keys()][:-1]
    acc = []
    for key in rkeys:
        acc.append(results_dict[key].acc_test)
    mean_acc_df.loc[comp[0], comp[1]] = np.mean(acc)
    std_acc_df.loc[comp[0], comp[1]] = np.std(acc)

    mcc = []
    for key in rkeys:
        mcc.append(results_dict[key].mcc_test)
    mean_mcc_df.loc[comp[0], comp[1]] = np.mean(mcc)
    std_mcc_df.loc[comp[0], comp[1]] = np.std(mcc)
    
    overall_res[comp] = results_dict

Fitting 10 folds for each of 399 candidates, totalling 3990 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 1586 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 3990 out of 3990 | elapsed:    7.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 0 completed
Fitting 10 folds for each of 399 candidates, totalling 3990 fits


[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 3990 out of 3990 | elapsed:    8.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 1 completed
Fitting 10 folds for each of 399 candidates, totalling 3990 fits


...

[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 3376 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 3990 out of 3990 | elapsed:   10.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 8 completed
Fitting 10 folds for each of 399 candidates, totalling 3990 fits


[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 2704 tasks      | elapsed:    7.1s


Run 9 completed
Mean test accuracy: 0.7513513513513514
Std dev test accuracy: 0.044900669529286896
Mean test MCC: 0.33301145552372485
Std dev test MCC: 0.13666008778324512


[Parallel(n_jobs=-1)]: Done 3990 out of 3990 | elapsed:    9.6s finished


Save the dataframes as CSVs

In [46]:
mean_mcc_df.to_csv(results_path+subfolder+'mean_mcc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_mcc_pairwise.csv')
mean_acc_df.to_csv(results_path+subfolder+'mean_acc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_acc_pairwise.csv')

In [47]:
pickle_dump(overall_res, results_path+subfolder+f"overall_res_{nbname}.p")

writing total_bytes=55219867...
writing bytes [0, 55219867)... done.


Print the results

In [48]:
mean_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.528133,0.593306,0.851568,0.685337,0.725643,0.775950,0.921601,0.635543,0.518299
Down,0.0,0.000000,0.714713,0.771487,0.663013,-0.051135,0.247415,0.710164,0.631107,0.476154
FXS-F,0.0,0.000000,0.000000,0.522707,0.524038,0.833950,0.843086,0.916324,0.299086,0.482091
FXS-M,0.0,0.000000,0.000000,0.000000,0.403202,0.827591,0.912526,0.869306,0.389471,0.648505
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.751142,0.822434,0.838419,0.053940,0.490525
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.045354,0.687646,0.754158,0.565447
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.744882,0.777842,0.689569
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.825965,0.811190
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333011
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
std_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.1845,0.173082,0.069608,0.172401,0.147602,0.090305,0.073968,0.088552,0.118281
Down,0.0,0.0000,0.136173,0.184880,0.157767,0.194446,0.215062,0.089345,0.188490,0.128923
FXS-F,0.0,0.0000,0.000000,0.216337,0.161083,0.132243,0.094826,0.076006,0.172117,0.232466
FXS-M,0.0,0.0000,0.000000,0.000000,0.135896,0.120091,0.067561,0.090764,0.196831,0.103166
NF1,0.0,0.0000,0.000000,0.000000,0.000000,0.186332,0.078067,0.107865,0.170594,0.115081
PW-UPD,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.201419,0.132120,0.126385,0.158029
PW-del,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117454,0.097257,0.186894
SMC15,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090974,0.060572
TSC,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136660
XXY,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [50]:
mean_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.833333,0.818519,0.922857,0.853571,0.877778,0.896429,0.968000,0.824444,0.782143
Down,0.0,0.000000,0.853333,0.913636,0.826667,0.513333,0.662500,0.846154,0.893939,0.756250
FXS-F,0.0,0.000000,0.000000,0.784615,0.757895,0.911111,0.915789,0.956250,0.744444,0.736842
FXS-M,0.0,0.000000,0.000000,0.000000,0.719231,0.919231,0.957692,0.945833,0.718605,0.833333
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.868421,0.905000,0.917647,0.656757,0.740000
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.542105,0.837500,0.911111,0.778947
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.870588,0.910811,0.840000
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.944118,0.905882
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.751351
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
std_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.067185,0.080209,0.036253,0.075677,0.064257,0.040564,0.029933,0.042687,0.051632
Down,0.0,0.000000,0.065320,0.062490,0.085375,0.079162,0.080039,0.048650,0.047432,0.058962
FXS-F,0.0,0.000000,0.000000,0.102050,0.078772,0.071146,0.053674,0.040020,0.052411,0.107863
FXS-M,0.0,0.000000,0.000000,0.000000,0.057177,0.052876,0.031949,0.037500,0.082844,0.050376
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.097759,0.041533,0.053913,0.066258,0.058310
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.074618,0.069597,0.044444,0.077352
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063355,0.038317,0.094340
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027747,0.028818
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044901
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Model 2: OvO SVM with linear kernel

Define a subfolder to store the results

In [36]:
subfolder = 'SVM_lin/'

Define the model

In [26]:
svc_lin = SVC(decision_function_shape='ovo', kernel='linear')
steps_lin = [('scaler', StandardScaler()),
     ('model',svc_lin)]
pipeline_lin = Pipeline(steps_lin)

Define the parameters for the grid search

In [27]:
lin_C_range = np.logspace(start=-10, stop=10, num=21, base=2)
lin_parameters = dict(model__C=lin_C_range)

Create dataframes to store the pairwise results

In [28]:
mean_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

mean_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

Train the pairwise models and store the results

In [29]:
overall_res = {}

for comp in comparisons:
    
    X_c, y_c = get_pairwise_data(comp[0], comp[1], X_all, y_all)
    results_dict = repeated_training(X_c, y_c, n_repeats, n_folds, lin_parameters, pipeline_lin, mcc_scorer, verbose=1)
    
    rkeys = [x for x in results_dict.keys()][:-1]
    acc = []
    for key in rkeys:
        acc.append(results_dict[key].acc_test)
    mean_acc_df.loc[comp[0], comp[1]] = np.mean(acc)
    std_acc_df.loc[comp[0], comp[1]] = np.std(acc)

    mcc = []
    for key in rkeys:
        mcc.append(results_dict[key].mcc_test)
    mean_mcc_df.loc[comp[0], comp[1]] = np.mean(mcc)
    std_mcc_df.loc[comp[0], comp[1]] = np.std(mcc)
    
    overall_res[comp] = results_dict

Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 0 completed
Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 1 completed
Fitting 10 folds for each of 21 candidates, totalling 210 fits


...

[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   25.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 7 completed
Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   34.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 8 completed
Fitting 10 folds for each of 21 candidates, totalling 210 fits
Run 9 completed
Mean test accuracy: 0.7648648648648649
Std dev test accuracy: 0.045305553011460026
Mean test MCC: 0.35919110403626714
Std dev test MCC: 0.15331099812011567


[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   31.6s finished


Save the dataframes as CSVs

In [38]:
mean_mcc_df.to_csv(results_path+subfolder+'mean_mcc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_mcc_pairwise.csv')
mean_acc_df.to_csv(results_path+subfolder+'mean_acc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_acc_pairwise.csv')

In [39]:
pickle_dump(overall_res, results_path+subfolder+f"overall_res_{nbname}.p")

writing total_bytes=25203179...
writing bytes [0, 25203179)... done.


Print the results

In [30]:
mean_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.470269,0.610356,0.799778,0.621718,0.747236,0.792601,0.930742,0.683951,0.385580
Down,0.0,0.000000,0.716811,0.820395,0.610985,0.100838,0.276769,0.681646,0.594008,0.454732
FXS-F,0.0,0.000000,0.000000,0.581077,0.464876,0.822116,0.876722,0.831305,0.214871,0.453674
FXS-M,0.0,0.000000,0.000000,0.000000,0.410229,0.804010,0.803268,0.828746,0.345389,0.680893
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.692715,0.745465,0.870746,0.139492,0.488836
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.032524,0.554437,0.599884,0.438051
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.799037,0.749749,0.657316
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.822676,0.761478
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359191
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
std_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.215844,0.166560,0.082660,0.105123,0.130535,0.114230,0.063405,0.148092,0.168708
Down,0.0,0.000000,0.133428,0.125504,0.170523,0.169913,0.264560,0.151004,0.157102,0.229618
FXS-F,0.0,0.000000,0.000000,0.144124,0.166229,0.108788,0.102138,0.218629,0.139271,0.180589
FXS-M,0.0,0.000000,0.000000,0.000000,0.197100,0.131471,0.129832,0.111132,0.083460,0.141545
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.129369,0.111773,0.127234,0.136505,0.078233
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.240705,0.155789,0.083404,0.149215
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141846,0.104483,0.127995
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117188,0.111329
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153311
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
mean_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.804167,0.82963,0.894286,0.828571,0.888889,0.903571,0.972000,0.844444,0.725000
Down,0.0,0.000000,0.86000,0.931818,0.800000,0.586667,0.662500,0.830769,0.887879,0.743750
FXS-F,0.0,0.000000,0.00000,0.815385,0.726316,0.905556,0.936842,0.912500,0.702778,0.721053
FXS-M,0.0,0.000000,0.00000,0.000000,0.723077,0.907692,0.903846,0.929167,0.702326,0.844444
NF1,0.0,0.000000,0.00000,0.000000,0.000000,0.836842,0.865000,0.935294,0.683784,0.735000
PW-UPD,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.484211,0.775000,0.855556,0.710526
PW-del,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.900000,0.902703,0.825000
SMC15,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.944118,0.882353
TSC,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.764865
XXY,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
std_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.056057,0.070662,0.044355,0.047380,0.054935,0.053093,0.025612,0.072350,0.067857
Down,0.0,0.000000,0.062893,0.046577,0.084327,0.058119,0.105327,0.082849,0.038450,0.098623
FXS-F,0.0,0.000000,0.000000,0.061538,0.084211,0.061111,0.051568,0.112500,0.049768,0.085029
FXS-M,0.0,0.000000,0.000000,0.000000,0.087368,0.060079,0.062611,0.045833,0.040011,0.067890
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.064245,0.059372,0.061414,0.043663,0.039051
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.102056,0.075000,0.020787,0.067606
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.069849,0.038602,0.064226
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035902,0.052613
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045306
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Model 3: Naive Bayes (learn prior)

See '<b>Pairwise Naive Bayes in R.ipynb</b>'

## Model 4: Naive Bayes (uniform prior)

See '<b>Pairwise Naive Bayes in R.ipynb</b>'

## Model 5: Standard Random Forest

Define a subfolder to store the results

In [52]:
subfolder = 'SRF/'

Define the model

In [53]:
rfc_naive = RandomForestClassifier()

Define the parameters for the grid search

In [54]:
n_features = X_all.shape[1]
default_mtry = np.sqrt(n_features)
rfc_parameters = {'max_features': [int(0.5*default_mtry), int(default_mtry), int(1.5*default_mtry), int(2*default_mtry), int(3*default_mtry)],
             'n_estimators': [2000], 'min_samples_leaf': [1,2, 3, 4, 5]}

Create dataframes to store the pairwise results

In [55]:
mean_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

mean_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

Train the pairwise models and store the results

In [56]:
overall_res = {}

for comp in comparisons:
    
    X_c, y_c = get_pairwise_data(comp[0], comp[1], X_all, y_all)
    results_dict = repeated_training(X_c, y_c, n_repeats, n_folds, rfc_parameters, rfc_naive, mcc_scorer, verbose=1)
    
    rkeys = [x for x in results_dict.keys()][:-1]
    acc = []
    for key in rkeys:
        acc.append(results_dict[key].acc_test)
    mean_acc_df.loc[comp[0], comp[1]] = np.mean(acc)
    std_acc_df.loc[comp[0], comp[1]] = np.std(acc)

    mcc = []
    for key in rkeys:
        mcc.append(results_dict[key].mcc_test)
    mean_mcc_df.loc[comp[0], comp[1]] = np.mean(mcc)
    std_mcc_df.loc[comp[0], comp[1]] = np.std(mcc)
    
    overall_res[comp] = results_dict

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.8min finished


Run 0 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.8min finished


Run 1 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


...

Run 7 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.7min finished
/Users/joefarrington/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Run 8 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.7min finished


Run 9 completed
Mean test accuracy: 0.7648648648648649
Std dev test accuracy: 0.04366349843622571
Mean test MCC: 0.3141370987558429
Std dev test MCC: 0.14994090555454456


Save the dataframes as CSVs

In [57]:
mean_mcc_df.to_csv(results_path+subfolder+'mean_mcc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_mcc_pairwise.csv')
mean_acc_df.to_csv(results_path+subfolder+'mean_acc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_acc_pairwise.csv')

In [58]:
pickle_dump(overall_res, results_path+subfolder+f"overall_res_{nbname}.p")

writing total_bytes=2272620117...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 2272620117)... done.


Print the results

In [59]:
mean_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.699018,0.634608,0.798169,0.650855,0.773943,0.715146,0.960404,0.647183,0.633886
Down,0.0,0.000000,0.735904,0.863890,0.639694,0.071143,0.234248,0.797836,0.424459,0.570718
FXS-F,0.0,0.000000,0.000000,0.448254,0.414886,0.840423,0.826678,0.882629,0.301275,0.701001
FXS-M,0.0,0.000000,0.000000,0.000000,0.525329,0.830461,0.880622,0.868525,0.514275,0.602858
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.871098,0.845406,0.835008,0.050939,0.615925
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.067895,0.603560,0.714657,0.754785
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.712462,0.647417,0.739116
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.827019,0.730856
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.314137
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
std_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.142198,0.096861,0.082000,0.173965,0.122074,0.055644,0.065698,0.060093,0.137746
Down,0.0,0.000000,0.157270,0.114547,0.173200,0.191440,0.212358,0.114055,0.229351,0.177625
FXS-F,0.0,0.000000,0.000000,0.189630,0.155624,0.119090,0.141585,0.149200,0.119260,0.145064
FXS-M,0.0,0.000000,0.000000,0.000000,0.168602,0.102595,0.090441,0.102701,0.112285,0.161712
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.090429,0.125373,0.079406,0.146929,0.161433
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.218964,0.226463,0.130256,0.163251
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115961,0.146274,0.108841
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086262,0.095234
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.149941
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
mean_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.891667,0.837037,0.897143,0.839286,0.900000,0.864286,0.984000,0.828889,0.832143
Down,0.0,0.000000,0.853333,0.950000,0.826667,0.593333,0.650000,0.892308,0.857576,0.800000
FXS-F,0.0,0.000000,0.000000,0.761538,0.705263,0.916667,0.910526,0.937500,0.772222,0.847368
FXS-M,0.0,0.000000,0.000000,0.000000,0.776923,0.923077,0.942308,0.945833,0.774419,0.814815
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.931579,0.920000,0.917647,0.718919,0.805000
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.468421,0.800000,0.897222,0.873684
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.852941,0.862162,0.865000
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.944118,0.864706
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.764865
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
std_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.046398,0.044444,0.040808,0.073627,0.052509,0.026726,0.026533,0.028197,0.057698
Down,0.0,0.000000,0.093333,0.042882,0.085375,0.069602,0.089268,0.061538,0.050797,0.078062
FXS-F,0.0,0.000000,0.000000,0.076537,0.078772,0.062113,0.070809,0.079057,0.027217,0.072356
FXS-M,0.0,0.000000,0.000000,0.000000,0.072569,0.045508,0.043001,0.041874,0.047717,0.072198
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.047368,0.064031,0.039019,0.027562,0.078899
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.109012,0.111102,0.044876,0.082213
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054233,0.059766,0.055000
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027747,0.045943
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043663
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Model 6: Balanced Random Forest

Define a subfolder to store the results

In [ ]:
subfolder = 'BRF/'

Define the model

In [64]:
rfc_bal = BalancedRandomForestClassifier()

Define the parameters for the grid search

In [65]:
n_features = X_all.shape[1]
default_mtry = np.sqrt(n_features)
rfc_parameters = {'max_features': [int(0.5*default_mtry), int(default_mtry), int(1.5*default_mtry), int(2*default_mtry), int(3*default_mtry)],
             'n_estimators': [2000], 'min_samples_leaf': [1,2, 3, 4, 5]}

Create dataframes to store the results

In [66]:
mean_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

mean_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

Train the pairwise models and store the results

In [67]:
overall_res = {}

for comp in comparisons:
    
    X_c, y_c = get_pairwise_data(comp[0], comp[1], X_all, y_all)
    results_dict = repeated_training(X_c, y_c, n_repeats, n_folds, rfc_parameters, rfc_bal, mcc_scorer, verbose=1)
    
    rkeys = [x for x in results_dict.keys()][:-1]
    acc = []
    for key in rkeys:
        acc.append(results_dict[key].acc_test)
    mean_acc_df.loc[comp[0], comp[1]] = np.mean(acc)
    std_acc_df.loc[comp[0], comp[1]] = np.std(acc)

    mcc = []
    for key in rkeys:
        mcc.append(results_dict[key].mcc_test)
    mean_mcc_df.loc[comp[0], comp[1]] = np.mean(mcc)
    std_mcc_df.loc[comp[0], comp[1]] = np.std(mcc)
    
    overall_res[comp] = results_dict

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.0min finished


Run 0 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   41.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.7min finished


Run 1 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


...

Run 7 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.4min finished


Run 8 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.4min finished


Run 9 completed
Mean test accuracy: 0.7243243243243243
Std dev test accuracy: 0.060192047165729956
Mean test MCC: 0.41809082735854897
Std dev test MCC: 0.118572978482041


Save the dataframes as CSVs

In [68]:
mean_mcc_df.to_csv(results_path+subfolder+'mean_mcc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_mcc_pairwise.csv')
mean_acc_df.to_csv(results_path+subfolder+'mean_acc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_acc_pairwise.csv')

In [69]:
pickle_dump(overall_res, results_path+subfolder+f"overall_res_{nbname}.p")

writing total_bytes=5117176009...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 3221225472)... done.
writing bytes [3221225472, 4294967296)... done.
writing bytes [4294967296, 5117176009)... done.


Print the results

In [70]:
mean_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.692885,0.763794,0.790452,0.610306,0.756966,0.701669,0.930675,0.595055,0.543382
Down,0.0,0.000000,0.773423,0.775328,0.696027,0.243239,0.425234,0.676684,0.477549,0.762408
FXS-F,0.0,0.000000,0.000000,0.572694,0.360243,0.793270,0.782474,0.808488,0.378848,0.623169
FXS-M,0.0,0.000000,0.000000,0.000000,0.407420,0.829034,0.895148,0.844220,0.408983,0.551469
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.830825,0.771259,0.858075,0.082483,0.655640
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.096515,0.555974,0.640281,0.859445
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.648803,0.620783,0.705415
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.795937,0.749632
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.418091
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [71]:
std_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.11951,0.099255,0.069152,0.156798,0.134143,0.144797,0.077412,0.136795,0.194077
Down,0.0,0.00000,0.135591,0.112438,0.105270,0.160915,0.164911,0.111888,0.111442,0.112045
FXS-F,0.0,0.00000,0.000000,0.127970,0.195228,0.100469,0.112880,0.113427,0.108170,0.115240
FXS-M,0.0,0.00000,0.000000,0.000000,0.171424,0.143972,0.075325,0.081575,0.110616,0.117787
NF1,0.0,0.00000,0.000000,0.000000,0.000000,0.129564,0.128939,0.148615,0.222646,0.115708
PW-UPD,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.250466,0.182513,0.080614,0.102353
PW-del,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176171,0.106749,0.122140
SMC15,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.144133,0.114880
TSC,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.118573
XXY,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [72]:
mean_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.866667,0.877778,0.891429,0.821429,0.881481,0.860714,0.972000,0.793333,0.792857
Down,0.0,0.000000,0.873333,0.904545,0.826667,0.600000,0.700000,0.830769,0.742424,0.875000
FXS-F,0.0,0.000000,0.000000,0.803846,0.678947,0.888889,0.884211,0.900000,0.691667,0.805263
FXS-M,0.0,0.000000,0.000000,0.000000,0.700000,0.919231,0.946154,0.933333,0.697674,0.781481
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.910526,0.875000,0.929412,0.540541,0.820000
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.547368,0.775000,0.833333,0.926316
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.829412,0.816216,0.850000
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.926471,0.876471
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.724324
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [73]:
std_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.055277,0.055060,0.035686,0.071429,0.065838,0.066720,0.031241,0.064444,0.084213
Down,0.0,0.000000,0.075719,0.047456,0.067987,0.084327,0.078062,0.057564,0.068098,0.062500
FXS-F,0.0,0.000000,0.000000,0.052876,0.092518,0.055556,0.061378,0.057282,0.075000,0.057895
FXS-M,0.0,0.000000,0.000000,0.000000,0.083913,0.065385,0.039223,0.033333,0.057907,0.056047
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.066782,0.068007,0.073471,0.128485,0.060000
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.122757,0.089268,0.048113,0.053674
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085040,0.057714,0.063246
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053023,0.055494
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060192
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Model 7: Weighted Random Forest

Define a subfolder to store the results

In [ ]:
subfolder = 'WRF/'

Define the model

In [56]:
rfc_weighted = RandomForestClassifier(class_weight = 'balanced_subsample')

Define the parameters for the grid search

In [57]:
n_features = X_all.shape[1]
default_mtry = np.sqrt(n_features)
rfc_parameters = {'max_features': [int(0.5*default_mtry), int(default_mtry), int(1.5*default_mtry), int(2*default_mtry), int(3*default_mtry)],
             'n_estimators': [2000], 'min_samples_leaf': [1,2, 3, 4, 5]}

Create dataframes to store the results

In [58]:
mean_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_mcc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

mean_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)
std_acc_df = pd.DataFrame(np.zeros(((len(labels)),(len(labels)))), index=labels, columns=labels)

Train the pairwise models and store the results

In [59]:
overall_res = {}

for comp in comparisons:
    
    X_c, y_c = get_pairwise_data(comp[0], comp[1], X_all, y_all)
    results_dict = repeated_training(X_c, y_c, n_repeats, n_folds, rfc_parameters, rfc_weighted, mcc_scorer, verbose=1)
    
    rkeys = [x for x in results_dict.keys()][:-1]
    acc = []
    for key in rkeys:
        acc.append(results_dict[key].acc_test)
    mean_acc_df.loc[comp[0], comp[1]] = np.mean(acc)
    std_acc_df.loc[comp[0], comp[1]] = np.std(acc)

    mcc = []
    for key in rkeys:
        mcc.append(results_dict[key].mcc_test)
    mean_mcc_df.loc[comp[0], comp[1]] = np.mean(mcc)
    std_mcc_df.loc[comp[0], comp[1]] = np.std(mcc)
    
    overall_res[comp] = results_dict

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.0min finished


Run 0 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.1min finished


Run 1 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


...

Run 7 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.3min finished


Run 8 completed
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.2min finished


Run 9 completed
Mean test accuracy: 0.7432432432432432
Std dev test accuracy: 0.04562687301657873
Mean test MCC: 0.3720863343803694
Std dev test MCC: 0.12802683414141985


Save the dataframes as CSVs

In [65]:
mean_mcc_df.to_csv(results_path+subfolder+'mean_mcc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_mcc_pairwise.csv')
mean_acc_df.to_csv(results_path+subfolder+'mean_acc_pairwise.csv')
std_mcc_df.to_csv(results_path+subfolder+'std_acc_pairwise.csv')

In [67]:
pickle_dump(overall_res, results_path+subfolder+f"overall_res_{nbname}.p")

writing total_bytes=2019155171...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2019155171)... done.


Print the results

In [60]:
mean_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.683801,0.656946,0.813999,0.607162,0.681757,0.701738,0.901521,0.629182,0.595577
Down,0.0,0.000000,0.799149,0.836088,0.687691,0.180609,0.216998,0.710817,0.542458,0.580258
FXS-F,0.0,0.000000,0.000000,0.517947,0.393889,0.872953,0.828971,0.782741,0.398660,0.667137
FXS-M,0.0,0.000000,0.000000,0.000000,0.490357,0.909274,0.782655,0.809025,0.552517,0.744445
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.743805,0.783502,0.846237,0.325447,0.667520
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.011252,0.570190,0.713780,0.764322
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.725435,0.672253,0.715163
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.849128,0.784341
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.372086
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
std_mcc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.140017,0.089134,0.079970,0.150967,0.148206,0.115750,0.089313,0.086095,0.164104
Down,0.0,0.000000,0.107209,0.094599,0.182873,0.163958,0.155888,0.177439,0.154279,0.132647
FXS-F,0.0,0.000000,0.000000,0.084669,0.264693,0.081170,0.110580,0.130563,0.173135,0.140905
FXS-M,0.0,0.000000,0.000000,0.000000,0.137884,0.068246,0.122829,0.070905,0.136256,0.100551
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.117791,0.096326,0.095472,0.144852,0.124012
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.208288,0.157847,0.105323,0.145895
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.177315,0.142343,0.141062
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.149950,0.182994
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128027
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
mean_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.870833,0.844444,0.905714,0.817857,0.859259,0.857143,0.960000,0.817778,0.817857
Down,0.0,0.000000,0.893333,0.936364,0.846667,0.606667,0.631250,0.853846,0.851515,0.781250
FXS-F,0.0,0.000000,0.000000,0.780769,0.684211,0.933333,0.910526,0.887500,0.763889,0.826316
FXS-M,0.0,0.000000,0.000000,0.000000,0.761538,0.957692,0.892308,0.920833,0.793023,0.877778
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.863158,0.885000,0.923529,0.759459,0.825000
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.787500,0.888889,0.873684
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.864706,0.872973,0.855000
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.952941,0.894118
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.743243
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [63]:
std_acc_df

,22q11DS,Down,FXS-F,FXS-M,NF1,PW-UPD,PW-del,SMC15,TSC,XXY
22q11DS,0.0,0.060237,0.043192,0.040507,0.066720,0.063721,0.059761,0.035777,0.044222,0.066720
Down,0.0,0.000000,0.061101,0.036364,0.089691,0.075719,0.081250,0.087368,0.045857,0.080283
FXS-F,0.0,0.000000,0.000000,0.030039,0.126754,0.041574,0.057895,0.067315,0.071632,0.070809
FXS-M,0.0,0.000000,0.000000,0.000000,0.059086,0.031949,0.059086,0.029167,0.059245,0.046995
NF1,0.0,0.000000,0.000000,0.000000,0.000000,0.058608,0.050249,0.045943,0.049023,0.060208
PW-UPD,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.094883,0.075000,0.041201,0.078772
PW-del,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083397,0.052755,0.072284
SMC15,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044019,0.086453
TSC,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045627
XXY,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
